## **Owner**: Ahmed Tarek Mohamed

# **Object Recognition for Real Guns and Water Guns.**

# **1. Import Libraries and Tools**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
import glob
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
from ipywidgets import Text, VBox

**Import Essential libraries and tools.**
* Imports the TensorFlow library and aliases it as tf.
* Imports specific modules from TensorFlow's Keras API for building neural
* network models (Sequential, Dense, Conv2D, MaxPool2D, Flatten).
* Imports the glob module for pattern matching file paths.
* Imports the OpenCV library for computer vision tasks, including image processing.
* Imports the train_test_split function from scikit-learn for splitting data into training and testing sets.
* Imports the NumPy library for numerical operations.
* Imports the matplotlib.pyplot module for creating visualizations.
* Imports the Image module from the Python Imaging Library (PIL) for image processing.
* Imports the requests module for making HTTP requests.
* Imports the BytesIO class from the io module for handling binary data as file-like objects.
* Imports specific widgets (Text and VBox) from the ipywidgets library for interactive user interfaces.

# **2. Class Definition for Image Classification**

In [ ]:
class ImageClassifier:
    def __init__(self, data_path, categories):
        self.data_path = data_path
        self.categories = categories
        self.model = None

    def load_images(self):
        data = []
        for idx, category in enumerate(self.categories):
            for file in glob.glob(f"{self.data_path}/{category}/*.jpg"):
                image = cv2.imread(file)
                if image is not None:
                    resized_image = cv2.resize(image, (50, 50))  # Change image size
                    data.append([resized_image, idx])
        return data

    def train_model(self, data):
        X = np.array([i[0] for i in data], dtype='float32') / 255.0
        y = np.array([i[1] for i in data])
        X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.2)
        X_val, X_test, y_val, y_test = train_test_split(X_other, y_other, test_size=0.5)
        self.model = Sequential([
            Conv2D(16, (3, 3), activation='relu', input_shape=(50, 50, 3)),
            MaxPool2D(2, 2),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPool2D(2, 2),
            Flatten(),
            Dense(64, activation='relu'),
            Dense(len(self.categories), activation='softmax')
        ])
        self.model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f"Accuracy: {accuracy:.2f}")

    def predict_image(self, image_url):
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((50, 50))
        img_array = np.expand_dims(np.array(img), axis=0) / 255.0
        prediction = self.model.predict(img_array)
        predicted_class = np.argmax(prediction)
        plt.imshow(img)
        plt.title(f"Predicted: {self.categories[predicted_class]}")
        plt.show()

**This code defines the ImageClassifier class, which encapsulates all functionalities related to image classification. It includes methods for loading images, training a convolutional neural network (CNN), and predicting the category of a new image. The class is initialized with a data path and a list of image categories.**

* The ImageClassifier class is designed for image classification tasks.
* The constructor (__init__) initializes the object with the provided data_path and categories.
* The load_images method reads and loads image data from the specified data_path for each category.
* The images are resized to 50x50 pixels and stored along with their corresponding category indices in the data list.
* The train_model method prepares the data for training, splits it into training, validation, and test sets, and creates a Convolutional Neural Network (CNN) model using the Keras Sequential API.
* The CNN model consists of convolutional layers with ReLU activation, max-pooling layers, and densely connected layers with softmax activation for classification.
* The model is compiled with the RMSprop optimizer and sparse categorical crossentropy loss function.
* It is trained on the training data for 20 epochs, and its performance is evaluated on the validation set.
* The trained model is stored in the self.model attribute.
* The predict_image method takes an image URL, downloads the image, resizes it to 50x50 pixels, and preprocesses it for prediction.
* The trained model predicts the class probabilities for the input image, and the predicted class is displayed along with the image using Matplotlib.

# **3. Main Execution ***

### TO TEST THE CODE USE IMAGE URL OF OBJECTS SIMILAR TO GUN

In [ ]:
# Constants
DATA_PATH = '/Datasets/WeaponDataset'
IMG_CATEGORIES = ['RealGun', 'NonRealGun']

# Main
classifier = ImageClassifier(DATA_PATH, IMG_CATEGORIES)
data = classifier.load_images()
classifier.train_model(data)

# User Input
def on_url_entered(change):
    classifier.predict_image(change.new)

url_input = Text(placeholder='Enter Gun URL', description='URL:')
url_input.observe(on_url_entered, names='value')
display(VBox([url_input]))

Epoch 1/20
24/24 [==============================] - 2s 57ms/step - loss: 0.7798 - accuracy: 0.5690 - val_loss: 0.6052 - val_accuracy: 0.7553
Epoch 2/20
24/24 [==============================] - 1s 50ms/step - loss: 0.5986 - accuracy: 0.6936 - val_loss: 0.5385 - val_accuracy: 0.7234
Epoch 3/20
24/24 [==============================] - 1s 49ms/step - loss: 0.5294 - accuracy: 0.7520 - val_loss: 0.6743 - val_accuracy: 0.5638
Epoch 4/20
24/24 [==============================] - 1s 59ms/step - loss: 0.4723 - accuracy: 0.7971 - val_loss: 0.5738 - val_accuracy: 0.7766
Epoch 5/20
24/24 [==============================] - 2s 87ms/step - loss: 0.4321 - accuracy: 0.8050 - val_loss: 0.4982 - val_accuracy: 0.7660
Epoch 6/20
24/24 [==============================] - 2s 85ms/step - loss: 0.3912 - accuracy: 0.8329 - val_loss: 0.6471 - val_accuracy: 0.6383
Epoch 7/20
24/24 [==============================] - 2s 78ms/step - loss: 0.3786 - accuracy: 0.8382 - val_loss: 0.6689 - val_accuracy: 0.7979
Epoch 8/20
24

- Defines the constant `DATA_PATH` with the path to the dataset directory (`'/content/drive/MyDrive/WeaponDataset'`).
- Defines the constant `IMG_CATEGORIES` with a list of image categories (`['RealGun', 'NonRealGun']`).

- Creates an instance of the `ImageClassifier` class named `classifier`, initialized with the dataset path and image categories.
  
- Loads images from the specified dataset path using the `load_images` method of the `classifier` object.

- Trains the image classification model using the loaded data with the `train_model` method of the `classifier` object.

- Defines a user input mechanism:
  - Creates a text input widget (`url_input`) with a placeholder and description.
  - Defines a function (`on_url_entered`) that is triggered when the user enters a URL. This function calls the `predict_image` method of the `classifier` object with the entered URL.
  - Associates the `on_url_entered` function with the `value` attribute of the `url_input` widget.

- Displays the text input widget (`url_input`) inside a vertical box (`VBox`) using the `display` function.

Note: Make sure to run this code in an environment where the `ImageClassifier` class is defined and the necessary libraries are imported. Also, the dataset path should be valid, and the dataset should contain images in the specified categories.

# **Summary**

The developed Python code provides a comprehensive solution for differentiating between images of real guns and water guns using a convolutional neural network (CNN). The process involves several key steps, including image loading, preprocessing, model training, and interactive user input handling for real-time classification.


* **Class Definition:** A custom class encapsulates the entire functionality. This class includes methods for loading images from a specified directory, preprocessing these images, training a CNN model on these images, and predicting the category of new images based on the trained model.

* **Model Training:** The CNN model is designed with convolutional layers, pooling layers, and dense layers. It is trained on a dataset categorized into 'RealGun' and 'NonRealGun'. The training involves resizing images to a uniform size and normalizing them. The model learns features from these images to distinguish between the two categories.

* **User Interaction:** An interactive component is built using ipywidgets. Users can input URLs of images, and the classifier predicts whether the image is a real gun or a water gun. This real-time interaction demonstrates the practical application of the mode